# Analysis of Ambiental Radioactivity by Matt Green
In this notebook, we analyse the events contained in the root files by Matt Green about ambiental radioactivity. He said that they simulated the full **232Th-chain, 238U-chain and 40K-chain**.

## First file: `A228_Z90` (*i.e Thorium's decay chain*)
Explaination of the name: A=228, Z=90 is **228Th**, an isotope of Thorium. Then, this file containes the simulation of the full decay-chain of Thorium.

First, we look for the atoms involved in this simulation.

In [1]:
import pandas as pd
import os

# Since input file is large, we have to distribute the workload
filepath1 = os.path.join("..", "Data", "output_AmbRad_A228_Z90_100000_part1.csv")
filepath2 = os.path.join("..", "Data", "output_AmbRad_A228_Z90_100000_part2.csv")
filepath3 = os.path.join("..", "Data", "output_AmbRad_A228_Z90_100000_part3.csv")
filepath4 = os.path.join("..", "Data", "output_AmbRad_A228_Z90_100000_part4.csv")
# create multiple dataframes
df1 = pd.read_csv(filepath1, index_col=False)
df2 = pd.read_csv(filepath2, index_col=False)
df3 = pd.read_csv(filepath3, index_col=False)
df4 = pd.read_csv(filepath4, index_col=False)
# concatenate the dfs in a single one and force the mem release
df = pd.concat([df1, df2, df3, df4], ignore_index=True)
del df1
del df2
del df3
del df4
df.head()

,PID,ParentTrackID,energydeposition,kineticenergy,time,x,y,z,px,py,pz,eventnumber,tracknumber,creatorprocess,parentnucleusPID
0,1000902280,0,0.000000,0.000000,0.000000e+00,-190.082,-3.79929,31.5809,0.000000,0.000000,-0.000000,0,1,primaryParticle,666
1,1000020040,2,0.000000,3.208800,3.594670e-04,-190.086,-3.80098,31.5845,-122.561000,-37.971500,86.418100,0,5,RadioactiveDecay,666
2,1000020040,2,3.208800,0.000000,2.197580e-03,-190.104,-3.80659,31.5973,-0.000000,-0.000000,0.000000,0,5,ionIoni,666
3,22,10,0.000000,0.300089,2.809210e+13,-190.050,-3.66345,31.5127,0.060967,0.262583,-0.131857,0,14,RadioactiveDecay,666
4,22,10,0.003178,0.246502,2.809210e+13,-187.493,7.35200,25.9812,-0.092471,0.099071,-0.205906,0,14,compt,666


In [3]:
print(sorted(df.PID.unique()))

[-11, 11, 22, 1000020040, 1000180360, 1000180380, 1000180400, 1000320740, 1000320760, 1000902280]


These are:
- `PID=-11` positrons
- `PID=11` electrons
- `PID=22` photons
- `PID=1000020040` Helium isotope 4He
- `PID=1000180360` Argon isotope 36Ar
- `PID=1000180380` Argon isotope 38Ar
- `PID=1000180400` Argon isotope 40Ar
- `PID=1000320740` Germanium isotope 74Ge
- `PID=1000320760` Germanium isotope 76Ge
- `PID=1000902290` Thorium isotope 229Th

There is something weird:
1. Why there is no 232Th? Why there is 229Th if in the 232Th-chain it is not reported?
1. Why there are no the elements involed in 232Th-chain (*e.g. Ra, Ac, Rn, Po, Pb, Bi, Tl*)?

Now, we observe the energy depositions.

In [19]:
edep_entries = df[df.energydeposition>0].energydeposition.reset_index()
edep_gby_event = df.groupby("eventnumber").energydeposition.sum().reset_index()
edep_gby_event.columns = ['eventnumber', 'cum_edep']    # to distinguish

print(edep_entries.energydeposition.describe())
print()
print(edep_gby_event.cum_edep.describe())

count    1.006648e+07
mean     3.360898e-02
std      2.358649e-01
min      2.077170e-10
25%      2.488150e-03
50%      1.371690e-02
75%      2.465830e-02
max      6.605850e+00
Name: energydeposition, dtype: float64

count    99293.000000
mean         3.407331
std          3.940104
min          0.000818
25%          0.965960
50%          1.968218
75%          4.200668
max         34.748797
Name: cum_edep, dtype: float64


The deposited energy is very low, in the order of few keV.

**[TODO]** Check that the unit of edep in MC out scheme is keV.

### Single event
Now, we observe a single event to understand the time and space distribution.

In [23]:
eventnumber = 666
event = df[df.eventnumber==eventnumber].sort_values("time")

#### Time distribution

In [26]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [25, 15]
plt.rcParams.update({'font.size': 30})